In [ ]:
# import pandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# import files for data_processing
# use google mount
# mount google drive (change to github later)
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

dataset1 = pd.read_csv('/content/drive/My Drive/cbp20cd.csv', encoding = 'latin-1', sep = ";")

Mounted at /content/drive


In [ ]:
dataset2 = pd.read_csv('/content/drive/My Drive/Aqueduct Water Risk.csv', encoding = 'latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12,16,20,24,32,36,40,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df1 = pd.DataFrame(dataset1)

In [ ]:
df2 = pd.DataFrame(dataset2)

In [ ]:
df1['2017 NAICS Code']
# should remove ---
df1['2017 NAICS Code'] = df1['2017 NAICS Code'].str.replace("-", "")
# should give 0 instead of NA
df1['2017 NAICS Code'] = df1['2017 NAICS Code'].fillna(0)
df1['2017 NAICS Code']

0         
1       11
2       21
3       22
4       23
        ..
9099     0
9100     0
9101     0
9102     0
9103     0
Name: 2017 NAICS Code, Length: 9104, dtype: object

In [ ]:
# take only certain rows
df_new = pd.DataFrame(df1[:9093])
df_new[" Number of Establishments "]
df_new["State FIPS"] = df_new['State FIPS'].astype(int)
df_new["State FIPS"]
# should remove ---
df_new[" Number of Establishments "] = df_new[" Number of Establishments "].str.replace(" ", "")
df_new[" Number of Establishments "] = df_new[" Number of Establishments "].astype(int)
df_new[" Employment "] = df_new[" Employment "].str.replace(" ", "")
df_new[" Employment "] = df_new[" Employment "].astype(int)
df_new[" Annual Payroll ($1,000) "] = df_new[" Annual Payroll ($1,000) "].str.replace(" ", "")
df_new[" Annual Payroll ($1,000) "] = df_new[" Annual Payroll ($1,000) "].astype(int)
df_new[" Annual Payroll ($1,000) "]

df_new["NAICS Description"]

#for col in df_new.columns:
#    print(col)

0                               Total for all sectors
1          Agriculture, forestry, fishing and hunting
2       Mining, quarrying, and oil and gas extraction
3                                           Utilities
4                                        Construction
                            ...                      
9088                Health care and social assistance
9089              Arts, entertainment, and recreation
9090                  Accommodation and food services
9091    Other services (except public administration)
9092                        Industries not classified
Name: NAICS Description, Length: 9093, dtype: object

In [ ]:
# Aggregate all values
merge = df_new[["State FIPS", " Number of Establishments ", '2017 NAICS Code', " Employment ",
                " Annual Payroll ($1,000) ", 'NAICS Description', "State"]].groupby(['State FIPS',
                                                       '2017 NAICS Code', 'NAICS Description','State']).agg('sum')
# remove grouping to get separate columns
merge = merge.reset_index()

In [ ]:
df11=merge.pivot(index='State', columns='NAICS Description', 
                 values = [" Employment ", " Number of Establishments ", " Annual Payroll ($1,000) "])
df11

df11[df11.columns[0:17]] = df11[df11.columns[0:17]].div(df11[' Employment ', 'Total for all sectors'], axis=0).round(2)
df11[df11.columns[18:21]] = df11[df11.columns[18:21]].div(df11[' Employment ', 'Total for all sectors'], axis=0).round(2)

df11[df11.columns[21:38]] = df11[df11.columns[21:38]].div(df11[" Number of Establishments ", 'Total for all sectors'], axis=0).round(2)
df11[df11.columns[39:42]] = df11[df11.columns[39:42]].div(df11[" Number of Establishments ", 'Total for all sectors'], axis=0).round(2)

df11[df11.columns[42:59]] = df11[df11.columns[42:59]].div(df11[' Annual Payroll ($1,000) ', 'Total for all sectors'], axis=0).round(2)
df11[df11.columns[60:63]] = df11[df11.columns[60:63]].div(df11[' Annual Payroll ($1,000) ', 'Total for all sectors'], axis=0).round(2)

# drop Puerto Rico and Hawaii
df11 = df11.drop(df11.index[[11, 39]])
df11 = df11.reset_index()
df11 = df11.fillna(0)

In [ ]:
df20 = pd.DataFrame(dataset2)
df21 = df20.loc[df20['gid_0'] == "USA"]
# keep only certain columns
df22 = df21[['name_1', 'area_km2', 'bws_score', 'bwd_score', 'iav_score', 'sev_score', 'gtd_score',
            'rfr_score', 'cfr_score', 'drr_score', 'ucw_score', 'cep_score', 'udw_score', 'usa_score', 'rri_score']]
# replace -9999 with NA
df22 = df22.replace(-9999, np.nan)
# fix this
# get means per state
#df22[df22.columns[2:15]] = df22[df22.columns[2:15]].fillna(df22[df22.columns[2:15]].mean())
means2 = df22[['name_1', 'bws_score', 'bwd_score', 'iav_score', 'sev_score', 'gtd_score',
            'rfr_score', 'cfr_score', 'drr_score',
               'ucw_score', 'cep_score', 'udw_score',
               'usa_score', 'rri_score']].groupby('name_1').agg('mean')
means2

#df22[df22.columns[2:15]] = df22[df22.columns[2:15]].fillna(means2)
#df22
#df22['bws_score'] = df22.groupby('name_1').transform(lambda x: x.fillna(x.mean()))
#df22['bws_score'] = df22['bws_score'].fillna(df22.groupby('name_1')['bws_score'].transform('mean'))
#df22['gtd_score'] = df22['gtd_score'].fillna(df22.groupby('name_1')['gtd_score'].transform('mean'))

df22[df22.columns[2:15]] = df22[df22.columns[2:15]].fillna(df22.groupby('name_1')[df22.columns[2:15]].transform('mean'))
df22.sort_values('name_1')

# Remove Hawaii
df22 = df22[df22.name_1 != "Hawaii"]

mean_tot = df22['gtd_score'].mean()

df22['gtd_score'] = df22['gtd_score'].fillna(mean_tot)
df22 = df22.sort_values('name_1')
df22['name_1'].unique()


array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [ ]:
# aggregate columns
merge2 = df22[df22.columns[0:15]].groupby('name_1').agg({'area_km2':'sum', 
                                                         'bws_score':'mean', 
                                                         'bwd_score':'mean',
                                                        'iav_score':'mean',
                                                        'sev_score':'mean',
                                                        'gtd_score':'mean',
                                                        'rfr_score':'mean',
                                                        'cfr_score':'mean',
                                                        'drr_score':'mean',
                                                        'ucw_score':'mean',
                                                        'cep_score':'mean',
                                                        'udw_score':'mean',
                                                        'usa_score':'mean',
                                                        'rri_score':'mean'})
merge2 = merge2.reset_index()

In [ ]:
#df11 = df11.append(merge2[merge2.columns[1:15]], sort = False)
merge2 = merge2.reset_index(drop=True)

horizontal_stack = pd.concat([merge2[merge2.columns[0:15]], df11], axis=1)

In [ ]:
# drop extra state column
horizontal_stack.drop(('State', ''), axis=1, inplace=True)
horizontal_stack.rename(columns = {'name_1':'State', 'area_km2': 'Sum_area',
                                   (' Employment ', 'Accommodation and food services'): "EmpAcco",
                                  (' Employment ', 'Administrative and support and waste'): "EmpAdmi",
                                  (' Employment ', 'Agriculture, forestry, fishing and hunting'): 'EmpAgr',
                                  (' Employment ', 'Arts, entertainment, and recreation'): 'EmpArts',
                                  (' Employment ', 'Construction'): 'EmpConst',
                                  (' Employment ', 'Educational services'): 'EmpEdu',
                                  (' Employment ', 'Finance and insurance'): 'EmpFin',
                                  (' Employment ', 'Health care and social assistance'): 'EmpHeal',
                                  (' Employment ', 'Industries not classified'): 'EmpNotClass',
                                  (' Employment ', 'Information'): 'EmpInform',
                                  (' Employment ', 'Management of companies and enterprises'): 'EmpMAnag',
                                  (' Employment ', 'Manufacturing'): 'EmpManu',
                                  (' Employment ', 'Mining, quarrying, and oil and gas extraction'): 'EmpMining',
                                  (' Employment ', 'Other services (except public administration)'): 'EmpOther',
                                  (' Employment ', 'Professional, scientific, and technical'): 'EmpProff_Tech_sci',
                                  (' Employment ', 'Real estate and rental and leasing'): 'EmpRealEs',
                                  (' Employment ', 'Retail trade'): 'EmpRetail',
                                  (' Employment ', 'Total for all sectors'): 'EmpTOT',
                                  (' Employment ', 'Transportation and warehousing'): 'EmpTransp',
                                  (' Employment ', 'Utilities'): 'EmpUtilities',
                                  (' Employment ', 'Wholesale trade'): 'EmpTrade'},inplace = True)

horizontal_stack.rename(columns = {(' Number of Establishments ', 'Accommodation and food services'): "EstAcco",
                                  (' Number of Establishments ', 'Administrative and support and waste'): "EstAdmi",
                                  (' Number of Establishments ', 'Agriculture, forestry, fishing and hunting'): 'EstAgr',
                                  (' Number of Establishments ', 'Arts, entertainment, and recreation'): 'EstArts',
                                  (' Number of Establishments ', 'Construction'): 'EstConst',
                                  (' Number of Establishments ', 'Educational services'): 'EstEdu',
                                  (' Number of Establishments ', 'Finance and insurance'): 'EstFin',
                                  (' Number of Establishments ', 'Health care and social assistance'): 'EstHeal',
                                  (' Number of Establishments ', 'Industries not classified'): 'EstNotClass',
                                  (' Number of Establishments ', 'Information'): 'EstInform',
                                  (' Number of Establishments ', 'Management of companies and enterprises'): 'EstMAnag',
                                  (' Number of Establishments ', 'Manufacturing'): 'EstManu',
                                  (' Number of Establishments ', 'Mining, quarrying, and oil and gas extraction'): 'EstMining',
                                  (' Number of Establishments ', 'Other services (except public administration)'): 'EstOther',
                                  (' Number of Establishments ', 'Professional, scientific, and technical'): 'EstProff_Tech_sci',
                                  (' Number of Establishments ', 'Real estate and rental and leasing'): 'EstRealEs',
                                  (' Number of Establishments ', 'Retail trade'): 'EstRetail',
                                  (' Number of Establishments ', 'Total for all sectors'): 'EstTOT',
                                  (' Number of Establishments ', 'Transportation and warehousing'): 'EstTransp',
                                  (' Number of Establishments ', 'Utilities'): 'EstUtilities',
                                  (' Number of Establishments ', 'Wholesale trade'): 'EstTrade'},inplace = True)

horizontal_stack.rename(columns = {(' Annual Payroll ($1,000) ', 'Accommodation and food services'): "PayAcco",
                                  (' Annual Payroll ($1,000) ', 'Administrative and support and waste'): "PayAdmi",
                                  (' Annual Payroll ($1,000) ', 'Agriculture, forestry, fishing and hunting'): 'PayAgr',
                                  (' Annual Payroll ($1,000) ', 'Arts, entertainment, and recreation'): 'PayArts',
                                  (' Annual Payroll ($1,000) ', 'Construction'): 'PayConst',
                                  (' Annual Payroll ($1,000) ', 'Educational services'): 'PayEdu',
                                  (' Annual Payroll ($1,000) ', 'Finance and insurance'): 'PayFin',
                                  (' Annual Payroll ($1,000) ', 'Health care and social assistance'): 'PayHeal',
                                  (' Annual Payroll ($1,000) ', 'Industries not classified'): 'PayNotClass',
                                  (' Annual Payroll ($1,000) ', 'Information'): 'PayInform',
                                  (' Annual Payroll ($1,000) ', 'Management of companies and enterprises'): 'PayMAnag',
                                  (' Annual Payroll ($1,000) ', 'Manufacturing'): 'PayManu',
                                  (' Annual Payroll ($1,000) ', 'Mining, quarrying, and oil and gas extraction'): 'PayMining',
                                  (' Annual Payroll ($1,000) ', 'Other services (except public administration)'): 'PayOther',
                                  (' Annual Payroll ($1,000) ', 'Professional, scientific, and technical'): 'PayProff_Tech_sci',
                                  (' Annual Payroll ($1,000) ', 'Real estate and rental and leasing'): 'PayRealEs',
                                  (' Annual Payroll ($1,000) ', 'Retail trade'): 'PayRetail',
                                  (' Annual Payroll ($1,000) ', 'Total for all sectors'): 'PayTOT',
                                  (' Annual Payroll ($1,000) ', 'Transportation and warehousing'): 'PayTransp',
                                  (' Annual Payroll ($1,000) ', 'Utilities'): 'PayUtilities',
                                  (' Annual Payroll ($1,000) ', 'Wholesale trade'): 'PayTrade'},inplace = True)


In [ ]:
# checking if okay

column_names = ["State", "Sum_area", "bws_score", 'bwd_score',
               'iav_score', 'sev_score', 'gtd_score', 'rfr_score', 'cfr_score',
               'drr_score','ucw_score','cep_score','udw_score','usa_score','rri_score',
               'EmpTOT', 'EmpAgr', 'EmpMining', 'EmpUtilities', 'EmpConst', 'EmpManu', 'EmpTrade',
               'EmpRetail', 'EmpTransp', 'EmpInform', 'EmpFin', 'EmpRealEs', 'EmpProff_Tech_sci',
               'EmpMAnag', 'EmpAdmi', 'EmpEdu', 'EmpHeal', 'EmpArts', 'EmpAcco', 'EmpOther', 'EmpNotClass',
               'EstTOT', 'EstAgr', 'EstMining', 'EstUtilities', 'EstConst', 'EstManu', 'EstTrade',
               'EstRetail', 'EstTransp', 'EstInform', 'EstFin', 'EstRealEs', 'EstProff_Tech_sci',
               'EstMAnag', 'EstAdmi', 'EstEdu', 'EstHeal', 'EstArts', 'EstAcco', 'EstOther', 'EstNotClass',
               'PayTOT', 'PayAgr', 'PayMining', 'PayUtilities', 'PayConst', 'PayManu', 'PayTrade',
               'PayRetail', 'PayTransp', 'PayInform', 'PayFin', 'PayRealEs', 'PayProff_Tech_sci',
               'PayMAnag', 'PayAdmi', 'PayEdu', 'PayHeal', 'PayArts', 'PayAcco', 'PayOther', 'PayNotClass']

horizontal_stack = horizontal_stack.reindex(columns=column_names)


In [ ]:
# print to csv file
horizontal_stack.to_csv(r'/content/drive/My Drive/Final_aggregated(NO_HAWAII).csv', index = False)
